In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 100)

data = pd.read_csv('./data_cleaned.csv')
# 確定着順が0の馬は何らかの理由で出走停止している馬のため、サンプルから除外
data = data[data['確定着順']!=0]

# horse number appended to レースIDを削除し、レース固有IDのみ保持
data['レースID'] = data['レースID'].astype(str)
data['レースID'] = data['レースID'].str[:-2]

# リークがあったり、事前にわからない情報を削除する
data = data.drop(columns=["馬印","レース印","入線着順", "賞金",'異常コード', '人気順', '走破時計', '補正タイム', '通過順1', '通過順2', '通過順3', '通過順4', '上がり3Fタイム','PCI','走破タイム','着差タイム','血統登録番号'])

# また、不要な情報（レース名）も同じく削除する
data = data.drop(columns=["レース名","馬主名"])

#時系列の順序重視するため、年を西暦に変換
data['年'] = "20" + data['年'].astype(str)
data['年'] = data["年"].str.pad(width=4, side='right', fillchar='0')

data.columns

C:\Users\toru7\AppData\Local\Temp\ipykernel_21548\3831992699.py:5: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./data/2000_2024_fulldata.csv',header=None, names=columns_list)


Index(['年', '月', '日', '回次', '場所', '日次', 'レース番号', 'クラスコード', '芝・ダ', 'トラックコード',
       '距離', '馬場状態', '馬名', '性別', '年齢', '騎手名', '斤量', '頭数', '馬番', '確定着順', '馬体重',
       '調教師', '所属地', '騎手コード', '調教師コード', 'レースID', '生産者名', '父馬名', '母馬名', '母の父馬名',
       '毛色', '生年月日', '単勝オッズ'],
      dtype='object')

In [ ]:
"""
競馬の着順確率予測用改良ベースラインスクリプト（LightGBM + GPU）
================================================================
* **改良点**: 過学習対策、ハイパーパラメータ調整、評価指標追加
"""

import os
import warnings
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import log_loss, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# ------------------------------
# 設定値（過学習対策で調整）
# ------------------------------
DATA_PATH        = "data2020_2024.csv"
ID_COL           = "レースID"
FINISH_COL       = "確定着順"
TEST_SIZE        = 0.2
RANDOM_STATE     = 42
NUM_BOOST_ROUND  = 3000  # 増加（early_stoppingで制御）
EARLY_STOPPING   = 150   # より厳格に
CARD_THRESHOLD   = 255

# ------------------------------
# 1. データ読み込み & 前処理
# ------------------------------
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

# dataが定義されていない場合の対応
if 'data' not in locals():
    print("dataが定義されていません。CSVファイルを読み込みます。")
    df = pd.read_csv(DATA_PATH)
else:
    df = data.copy()

df[ID_COL] = df[ID_COL].astype(str)
df = df.dropna(subset=[FINISH_COL])
df[FINISH_COL] = pd.to_numeric(df[FINISH_COL], errors="coerce").astype(int)

print(f"データサイズ: {df.shape}")
print(f"レース数: {df[ID_COL].nunique()}")
print(f"着順分布:\n{df[FINISH_COL].value_counts().sort_index()}")

# ------------------------------
# 2. ラベル生成
# ------------------------------
label_map = {1: 0, 2: 1, 3: 2}
df["label"] = df[FINISH_COL].map(label_map).fillna(3).astype(int)

print(f"ラベル分布:\n{df['label'].value_counts().sort_index()}")

# ------------------------------
# 3. 特徴量選定（改良版）
# ------------------------------
leak_cols = {FINISH_COL, "入線着順", "賞金", "label", ID_COL}
feature_cols = [c for c in df.columns if c not in leak_cols]

print(f"使用する特徴量数: {len(feature_cols)}")
print(f"使用する特徴量: {feature_cols}")
# カテゴリ変数の処理（改良版）
categorical_cols = [
    c for c in feature_cols
    if df[c].dtype == "object" or df[c].dtype.name == "category"
]

# 高カーディナリティ対策
numeric_high_card_cols = []
for c in categorical_cols:
    nunique = df[c].nunique(dropna=False)
    if nunique > CARD_THRESHOLD:
        print(f"高カーディナリティ変数を数値化: {c} (unique: {nunique})")
        # より適切なエンコーディング
        le = LabelEncoder()
        df[c] = le.fit_transform(df[c].astype(str))
        numeric_high_card_cols.append(c)

categorical_cols = [c for c in categorical_cols if c not in numeric_high_card_cols]
for c in categorical_cols:
    df[c] = df[c].astype("category")

print(f"カテゴリ変数数: {len(categorical_cols)}")
print(f"数値化した高カーディナリティ変数数: {len(numeric_high_card_cols)}")

X = df[feature_cols]
y = df["label"]
groups = df[ID_COL]

# ------------------------------
# 4. 学習 / 検証データ分割（レース単位）
# ------------------------------
splitter = GroupShuffleSplit(n_splits=1, test_size=TEST_SIZE, random_state=RANDOM_STATE)
train_idx, val_idx = next(splitter.split(X, y, groups=groups))

X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

print(f"訓練データ: {len(X_train)} 行")
print(f"検証データ: {len(X_val)} 行")

# ------------------------------
# 5. LightGBM Dataset 作成
# ------------------------------
lgb_train = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_cols, free_raw_data=False)
lgb_val = lgb.Dataset(X_val, label=y_val, categorical_feature=categorical_cols, free_raw_data=False)

# ------------------------------
# 6. 学習パラメータ（過学習対策で調整）
# ------------------------------
params = {
    "objective": "multiclass",
    "num_class": 4,
    "metric": "multi_logloss",
    "learning_rate": 0.03,      # 低下（0.05 → 0.03）
    "num_leaves": 64,           # 大幅削減（255 → 64）
    "feature_fraction": 0.7,    # 削減（0.8 → 0.7）
    "bagging_fraction": 0.7,    # 削減（0.8 → 0.7）
    "bagging_freq": 1,
    "max_depth": 8,             # 制限追加（-1 → 8）
    "min_data_in_leaf": 50,     # 追加
    "min_gain_to_split": 0.1,   # 追加
    "lambda_l1": 0.1,           # L1正則化追加
    "lambda_l2": 0.1,           # L2正則化追加
    "max_bin": 255,
    "seed": RANDOM_STATE,
    "device_type": "gpu",
    "verbose": -1,
}

# ------------------------------
# 7. 学習
# ------------------------------
print("学習開始...")
model = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_train, lgb_val],
    valid_names=["train", "valid"],
    num_boost_round=NUM_BOOST_ROUND,
    callbacks=[lgb.early_stopping(EARLY_STOPPING), lgb.log_evaluation(100)],
)

print(f"最適反復回数: {model.best_iteration}")

# ------------------------------
# 8. 予測 & 評価
# ------------------------------
# 全データで予測
proba_all = model.predict(X, num_iteration=model.best_iteration)
proba_train = model.predict(X_train, num_iteration=model.best_iteration)
proba_val = model.predict(X_val, num_iteration=model.best_iteration)

# 評価指標計算
train_logloss = log_loss(y_train, proba_train)
val_logloss = log_loss(y_val, proba_val)
all_logloss = log_loss(y, proba_all)

print(f"\n=== 評価結果 ===")
print(f"訓練データ multi_logloss: {train_logloss:.4f}")
print(f"検証データ multi_logloss: {val_logloss:.4f}")
print(f"過学習度合い: {val_logloss - train_logloss:.4f}")
print(f"全データ multi_logloss: {all_logloss:.4f}")

# 精度評価
pred_train = np.argmax(proba_train, axis=1)
pred_val = np.argmax(proba_val, axis=1)

print(f"\n訓練精度: {accuracy_score(y_train, pred_train):.4f}")
print(f"検証精度: {accuracy_score(y_val, pred_val):.4f}")

# 詳細な分類レポート
print(f"\n=== 検証データ分類レポート ===")
print(classification_report(y_val, pred_val, 
                          target_names=['1着', '2着', '3着', '圏外']))

# ------------------------------
# 9. 特徴量重要度
# ------------------------------
importance = model.feature_importance(importance_type='gain')
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importance
}).sort_values('importance', ascending=False)

print(f"\n=== TOP 10 重要特徴量 ===")
print(feature_importance.head(10))

# ------------------------------
# 10. 結果保存
# ------------------------------
proba_df = pd.DataFrame(proba_all, columns=["P1", "P2", "P3", "Pout"], index=df.index)
result = pd.concat([df[[ID_COL, "馬名"]], proba_df], axis=1)
result.to_csv("predictions_improved_lightgbm.csv", index=False)
print("\n→ predictions_improved_lightgbm.csv に確率を保存しました")

# モデル保存
model.save_model("lightgbm_model.txt")
print("→ lightgbm_model.txt にモデルを保存しました")

データサイズ: (1209558, 34)
レース数: 86343
着順分布:
確定着順
1     86430
2     86371
3     86377
4     86330
5     86324
6     86237
7     85995
8     85124
9     83210
10    80034
11    75601
12    70032
13    62623
14    54799
15    45732
16    34362
17     7909
18     6068
Name: count, dtype: int64
ラベル分布:
label
0     86430
1     86371
2     86377
3    950380
Name: count, dtype: int64
使用する特徴量数: 32
使用する特徴量: ['年', '月', '日', '回次', '場所', '日次', 'レース番号', 'レース名', 'クラスコード', '芝・ダ', 'トラックコード', '距離', '馬場状態', '馬名', '性別', '年齢', '騎手名', '斤量', '頭数', '馬番', '馬体重', '調教師', '所属地', '騎手コード', '調教師コード', '馬主名', '生産者名', '父馬名', '母馬名', '母の父馬名', '毛色', '生年月日']
高カーディナリティ変数を数値化: レース名 (unique: 4321)
高カーディナリティ変数を数値化: 馬名 (unique: 117192)
高カーディナリティ変数を数値化: 騎手名 (unique: 935)
高カーディナリティ変数を数値化: 調教師 (unique: 1121)
高カーディナリティ変数を数値化: 馬主名 (unique: 4871)
高カーディナリティ変数を数値化: 生産者名 (unique: 4696)
高カーディナリティ変数を数値化: 父馬名 (unique: 2330)
高カーディナリティ変数を数値化: 母馬名 (unique: 34935)
高カーディナリティ変数を数値化: 母の父馬名 (unique: 2997)
カテゴリ変数数: 6
数値化した高カーディナリティ変数数: 9
訓練データ: 968126 行